In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle

In [1]:
# Pull links for Nielsen ratings for 2014-17
url_17 = 'https://docs.google.com/spreadsheets/d/1ToHTY-SGH1wLEo31kKz9TOE5PsdpBE20kqtr1BhJBVA/pubhtml/sheet?headers=false&gid=141071556&range=A1%3AG100'
url_16 = 'https://docs.google.com/spreadsheets/d/1fDDhVVuLxhWiZYf0rhHouQreu36bCgk6gBcUzJONvaw/pubhtml/sheet?headers=false&gid=199072466&range=A1%3AG100'
url_15 = 'https://docs.google.com/spreadsheets/d/1pH5Y9hpycrTjdNptz38lM36nae-vswX7EwVAsSrBwfA/pubhtml/sheet?headers=false&gid=24&range=A1:G120'
url_14 = 'https://docs.google.com/spreadsheets/d/15w78kvpAodRlJ2ZO7eXI-oeRu1iROQjHhTW1Ha5GIHo/pubhtml/sheet?headers=false&gid=24&range=A1%3AG57&output=html&rm=minimal'

In [32]:
urls = [url_16, url_15, url_14]

In [45]:
# Function to scrape Nielsen data using BeautifulSoup
def get_shows(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "html5lib")
    big_table = soup.find(name="table", attrs={"class":"waffle"})
    table_body = big_table.find("tbody")
    table_rows = table_body.find_all("tr")
    all_rows = []
    for tr in table_rows:
        if len(tr.find_all("td")) > 2:
            all_rows.append(tr.find_all("td"))
        
    shows=[]
    for row in all_rows:
        if row[1].find('a'):
            one_show = {
                "title": row[1],
                "network": row[2],
                "viewers_millions": row[5],
                "demo_rating": row[3],
                "demo_yoy": row[4],
                "viewers_yoy": row[6],
                "status_year" : soup.find("title").text.split(' ')[0].strip()
            }
            shows.append(one_show)
    for each in shows:
        for key, value in each.items():
            if value != None and key != "status_year":
                each[key] = value.text
    return shows



In [46]:
# Specifying episodes to pull for 2017 scrape to avoid duplicates
shows_17 = get_shows(url_17)[0:193]

In [47]:
# Run scrape function for remaining years
all_shows = []
for url in urls:
    all_shows += get_shows(url)


In [48]:
# Combine all scraped data
all_shows = all_shows + shows_17

In [49]:
# Save data in pickle file for use in modeling workbook
with open('tvseries_data_2.pkl', 'wb') as picklefile:
    pickle.dump(all_shows, picklefile)